# 量子近似多目的最適化（QMOO）
#### 参考リンク: [Quantum Approximate Multi-Objective Optimization](https://www.nature.com/articles/s43588-025-00873-y)

量子近似多目的最適化（Quantum Approximate Multi-Objective Optimization、QMOO）は、近い将来における量子アドバンテージの有力な候補です。これは、古典的な手法を上回る性能を発揮する可能性を持ちながら、比較的小規模な量子計算と古典的な後処理を組み合わせるだけで実行できるためです。QMOOは、多目的最適化（Multi-Objective Optimization、MOO）に対する近似解を求める手法です。すなわち、$x\in X \subseteq \mathbb{R}^{n}$という変数の割り当てを計算し、 $m$ 個の目的関数 $f_{i}: X \mapsto \mathbb{R}$
の出力が同時に最適化されるようにします。例えば、

\begin{equation}
\max_{x\in X} f(x)=(f_{1}(x), ..., f_{m}(x))
\end{equation}

多目的最適化（MOO）の身近な例として、私たちは日常生活における旅行を挙げることができます。旅行では、費用（例えば燃料消費量やチケット価格）、移動距離、所要時間、環境負荷といった複数の目的が存在し、それらを組み合わせて最適化する必要があります。
通常、これらすべての目的を同時に最適化することは不可能です。すなわち、
$x\in X$から$x' \in X$へと変更すると、ある目的関数 $f_i$ は改善される一方で、別の目的関数 $f_j$ は悪化してしまう場合があります（例えば、より安いチケットは移動時間が長くなることが多いです）。そのため、多目的最適化では、パレートフロント$F$ を定義する点の集合 $x \in P\subset X$ を計算する必要があります。

$$
F = \left\{ (f_1(x), \dots, f_m(x)) \;\middle|\;
\nexists x' \in X \;\text{such that}\;
\forall i\; f_i(x') \geq f_i(x) \;\wedge\;
\exists j\; f_j(x') > f_j(x)
\right\}
$$

パレートフロントには、多目的最適化（MOO）アルゴリズムによって得られた最良の解が含まれます。すなわち、ある$x \in X$ に対して、少なくとも一つの目的関数$f_{j}(x')$の値が $f_{j}(x)$ よりも厳密に良く、かつ他のすべての目的関数 $f_{i}(x')$の値が $f_{i}(x)$より悪化しないような$x' \in X$ は存在しません。

次の例の図では、ヒューリスティックな多目的最適化手法によって、変数の割り当て $x \in X$ の集合が求められており、その評価値 $(f_{1}, f_{2})$ がそれぞれ x 軸と y 軸に示されています。ここでは、両方の目的関数を最大化するものとします。
この結果におけるパレートフロントは、線で結ばれた点として描かれています。パレートフロント上の各点は、他のすべてのパレートフロント上の点と比較して、一方の次元では優れているものの、もう一方の次元では劣っています。
一方、パレートフロント上にないすべての点は、少なくとも一つのパレートフロント上の点によって支配されており、すなわち両方の次元において劣っています。

In [ ]:
from qmoo_plot import plot_pareto
plot_pareto()

このノートブックでは、量子近似最適化アルゴリズム（Quantum Approximate Optimization Algorithm、QAOA）と古典的な後処理を用いて、多目的 Max-Cut 問題 に対するパレートフロントを計算します（詳細は [Quantum approximate multi-objective optimization](https://www.nature.com/articles/s43588-025-00873-y) を参照してください）。具体的には、次の内容を扱います。

* 多目的 Max-Cut に適したアンサッツ回路の準備
* アンサッツ回路からのサンプリングによって、パレートフロント上の点を探索する方法
* アンサッツ回路から得られたサンプルに基づき、パレートフロントを決定するための古典的後処理
* 得られたパレートフロントの品質を定量化するための指標

説明の都合上、本ノートブックでは $m=2$  次元の最適化、すなわち 2 つのグラフに対する Max-Cut 値の最大化 に焦点を当てて説明しますが、実際のチャレンジでは $m=3$ 個の目的関数を扱います。

## Python imports

本ノートブックでは、QDC チャレンジ用リポジトリのルートディレクトリにおいて `pip install -r requirements.txt` を実行することで初期化された Python 環境から作業を開始するものとします。

In [ ]:
from qmoo_files import load_problem
from qmoo_plot import plot_graphs, plot_pareto_hv

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerOptions, SamplerV2
from qiskit.circuit import ParameterVector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes.routing.commuting_2q_gate_routing import SwapStrategy
from qiskit.transpiler import CouplingMap
from qiskit.circuit.library.standard_gates.equivalence_library import _sel
from qiskit_aer.primitives import SamplerV2 as SamplerAerV2

from qiskit_addon_opt_mapper.applications import Maxcut
from qiskit_addon_opt_mapper.translators import to_ising
from qopt_best_practices.transpilation import UnrollBoxes
from qopt_best_practices.transpilation.generate_preset_qaoa_pass_manager import generate_preset_qaoa_pass_manager

from matplotlib import pyplot as plt
from moocore import hypervolume, filter_dominated


import networkx as nx
import numpy as np

In [ ]:
# Use the values from your IBM Quantum account
"""
instance = "<FILL IN YOUR INFO>"
token    = "<FILL IN YOUR INFO>"

service = QiskitRuntimeService(
    name  = "qdc-2025",
    channel  = "ibm_cloud",
    instance = instance,
    token    = token )
"""

#backend = service.backend("ibm_boston")
#backend = service.backend("ibm_kingston")
backend = service.backend("ibm_pittsburgh")

print(f"Using backend {backend.name}")

Grader functionをインポートし、ポイントを割り当てるチーム名を送信します。

In [ ]:
from qc_grader.challenges.qdc_2025 import qdc25_lab5, grade_lab5_ex1, grade_lab5_ex2, grade_lab5_ex3, grade_lab_5_ex4
qdc25_lab5.submit_name(YOUR_TEAM_NAME)

## QAOA を用いた多目的最適化

このチャレンジでは、多目的最適化（MOO）問題を解くための量子アプローチを、重み付き和法（Weighted Sum Method、WSM）と呼ばれる古典的手法に着想を得て構築します。
WSM では、多目的最適化問題 $\max_{x\in X} f_{i}(x)$ に対して、次のような単一の目的関数
\begin{equation}
f_{c}(x)= \sum_{i=1}^{m} c_{i} f_{i}(x)
\end{equation}
を構成します。
その後、$f_{c}(x)$ を最適化する、すなわち $f_{c}(x)$ が最大となるような変数 $x$ の割り当てを求めることを、通常の単一目的最適化手法によって行います。
重みベクトル  $c\in [0, 1]^m$ を適切に離散化することで、WSM により、目的関数 $f_i$ によって定義される多目的最適化問題の パレートフロント を求めることが可能になります。
重みベクトル $c$ を離散化する代替手法として、 $\sum_{i}c_{i}=1$を満たすランダムな重みベクトル $c$ の集合を選び、それぞれについて$f_c(x)$を定義する方法もあります。

### ハイパーボリューム ― パレートフロントの品質を定量化する指標
ハイパーボリュームは、パレートフロント（青い点） の品質を定量的に評価するために広く用いられている指標です。
目的関数が 2 つの場合、ハイパーボリュームは、参照点に対して求められたパレートフロントの「下側」に広がる面積として定義され、下図では緑色の領域として示されています（目的関数が 2 つより多い場合、この面積は体積になります）。
この例では、参照点は単純に $(0, 0)$ としています。より一般的な場合には、参照点は通常、各目的関数の下限値として定められます。例えば$(\min f_{1}(x), \min f_{2}(x))$ のように設定されます。
このような下限値は、多くの最適化問題に対して 線形計画（LP）緩和を用いることで効率的に求めることができます。


In [ ]:
plot_pareto_hv(hv=True, second=False)

同一の多目的最適化（MOO）問題に対して 2 つのパレートフロント $P_{1}, P_{2}$ を比較する場合、より大きなハイパーボリュームを持つパレートフロント ($P_1$) に含まれる解は、より小さなハイパーボリュームを持つパレートフロント ($P_2$) に含まれる解を支配します。

$\begin{equation}
\forall y \in P_2 \;\forall i \in \{1, \dots, m\} \;\exists x \in P_1 \;\text{such that}\; f_i(x) \geq f_i(y) \;\wedge\; \exists j \in \{1, \dots, m\} \; f_j(x) > f_j(y),
\end{equation}$

すなわち、パレートフロント $P_2$ に含まれる任意の点に対して、パレートフロント $P_1$ の中には、少なくとも 1 つの目的関数の値が厳密に大きく、かつ他のすべての目的関数の値が少なくとも同等であるような点が必ず存在します。
さらに、ハイパーボリュームが大きいほど、解の多様性が高く、かつ真のパレートフロントとの距離が小さいことを意味します。
次の図では、緑色およびオレンジ色のハイパーボリュームと、それぞれに対応するパレートフロントが、オレンジ色の×印および青色の点として示されています。オレンジ色の×印で示されたパレートフロントは、青色の点で示されたパレートフロントによって支配されています。


In [ ]:
plot_pareto_hv(hv=True, second=True)

## 0. ランダムなMOO Max-Cut 問題の生成
まず、ノードが $42$ 個のグラフ上で $m=3$ のMOO最大カット問題を生成することから始めます。

In [ ]:
# Note: we show how to determine `qaoa_params`, `upper_bounds`, `lower_bounds` in subsequent cells of the notebook 

moo_graphs, qaoa_params, upper_bound, lower_bound = load_problem("./instances/soa_42q/")
n_obj = len(moo_graphs)

In [ ]:
# plot the target max-cut graphs
plot_graphs(moo_graphs)

$n$ ノードからなる Max-Cut の多目的最適化（MOO）問題は、$x\in \{0, 1\}^{n}$という割り当てを見つけることから構成されます。この割り当てによって、上に示した各グラフにおいて誘導されるカットが最大化されることを目的とします。すなわち、グラフ$G=\{g_{0}, g_{1}, g_{2}\}$に対して、$x_{i}=1$ であるノードと $x_{j}=0$ であるノードを結ぶ辺の重みの総和が最大となるようにします。

### 1. 最大カットMOO用のQAOAアンサッツ回路を生成する

以下はQAOAの簡単な紹介です。理論については論文[A Quantum Approximate Optimization Algorithm](https://arxiv.org/abs/1411.4028)を読み、[Quantum approximate optimization algorithm](https://quantum.cloud.ibm.com/docs/tutorials/quantum-approximate-optimization-algorithm)チュートリアルや[Utility-scale QAOA](https://quantum.cloud.ibm.com/learning/courses/quantum-computing-in-practice/utility-scale-qaoa)コースを実践的にお試しください。
QAOAでは、$n$変数とコスト行列$Q$を持つ二次制約なし二項最適化問題(QUBO)を、イジングハミルトニアン$H_C$の基底状態を求めることで解きます。

\begin{equation}
H_{C} = -\sum_{i < j} J_{ij} \sigma^{z}_{i} \sigma^{z}_{j} - \sum_i h_i \sigma^{z}_{i},
\end{equation}


ここで$J_{ij}$と$h_{i}$はコスト行列$Q$から得られます。

この基底状態は以下を準備し、測定することによって決定されます

\begin{equation}
|\psi(\beta, \gamma)\rangle = \prod_{k=1}^{p} e^{-i\beta_k H_X} e^{-i\gamma_k H_C} |+\rangle,
\end{equation}

ここで$H_x= -\sum_{i} \sigma^{x}_{i}$ は混合ハミルトニアン、$\beta、\gamma$ は変分パラメータ、$p$ は QAOA 層の数です。ノード$u, v$間の辺 $e=\{u, v\}$ の重み$w_{uv}$ を持つグラフの最大カットを求めるには、以下の設定が必要です。


\begin{equation}
H_{C} = \sum_{u,v} (1 - Z_{u}Z_{v}),
\end{equation}

一定のエネルギーオフセットで最大で $\frac{1}{2}\sum_{u,v}w_{uv}$ までとして。
上記の和の項は、ノード$u, v$を表す量子ビットに$\texttt{RZZ}(w_{u, v} \gamma)$ゲートをもたらします。混合ハミルトニアンは、各量子ビット上の個別の$\texttt{RX}(2\beta)$単一量子ビットゲートで表現できます。

WSM(重み付け和法)アプローチで求められる個別グラフ $g_{i}\in G$ 上の最大カット問題の和を解く方法

\begin{equation}
\max_{x\in X} \sum_{i=1}^{m} c_{i} f_{i}(x),
\end{equation}

は、グラフ$g_{i} \in G$のすべての辺を含む結合グラフ$g$上の最大カット問題を解くことで表現できます。ここでグラフ$g$の辺$e$の辺の重み$w_{g,e}$は次のように決まります。

\begin{equation}
w_{g, e} = \sum_{i} c_{i} \cdot w_{g_{i}, e}
\end{equation}

したがって、多目的重みベクトル$c$を持つ適切なパラメータ化されたアンサッツ回路を$p=2$層に対して生成できます。
QAOAレイヤーの最適数$p$は、ターゲットQPUのエラー率、MOOで考慮される最適化問題、そして最大カットの入力グラフのサイズや構造など問題インスタンスに依存します。

In [ ]:
from qopt_best_practices.circuit_library import annotated_qaoa_ansatz

p_layers = 2
maxcuts = [Maxcut(g) for g in moo_graphs]
cost_ops = [to_ising(mc.to_optimization_problem())[0].simplify() for mc in maxcuts]
c_vec = ParameterVector("c", len(cost_ops))
sum_cost_op = sum(c_vec[idx] * hc for idx, hc in enumerate(cost_ops)).simplify()

# Note that we created an ansatz circuit where the individual QAOA cost and QAOA mixer layers are annotated for improved transpilation
qc = annotated_qaoa_ansatz(sum_cost_op, reps=p_layers)
qc.measure_all(inplace=True)
qc.draw(idle_wires=False,fold=-1)


次に、QAOA のアンサッツ回路を学習させる必要があります。すなわち、重みベクトル $c$ を単位値に固定したうえで、適切なパラメータ $\beta、\gamma$
を決定します。これまでの研究から、QAOA のパラメータ $\beta、\gamma$ は、特定のグラフインスタンスに対して一度最適化すると、多くの他のグラフに対しても有効に機能することが知られています。そのため、単位ベクトルに対して最適化された 1 組の QAOA パラメータ $\beta、\gamma$ が、任意の重みベクトル $c$ に対してもうまく機能すると期待されます。本ケースでは、$c=(1/3, 1/3, 1/3)$ を用いて最適化を行います。
パラメータの決定には、近似的な量子シミュレータと、COBYLA などの反復型最適化アルゴリズムを使用します。パラメータを求める他の方法としては、パラメータ転移則（parameter transfer rules）による外挿や、線形ランプ（linear ramps）のような静的パラメータスケジュールなどが挙げられます。

> **注:** 量子ビット数や QAOA のレイヤー数が多い場合、このステップには時間がかかることがあります。

In [ ]:
load_params_file = True

if load_params_file:
    # load low-effort QAOA parameters from file, see `else` branch for how to get advanced QAOA parameters
    params = qaoa_params[str(p_layers)]
    ansatz_qc = qc.assign_parameters(params)
else:
    # Train QAOA using the QAOA training pipeline https://github.com/qiskit-community/qaoa_training_pipeline
    from qaoa_training_pipeline.training import ScipyTrainer
    from qaoa_training_pipeline.evaluation import MPSEvaluator, StatevectorEvaluator
    
    trainer = ScipyTrainer(MPSEvaluator(bond_dim_circuit=20), minimize_args={"options": {"maxiter": 500}})
    # Use an exact simulator for small-scale quantum computations
    # trainer = ScipyTrainer(StatevectorEvaluator())

    # parameter initialization according to https://arxiv.org/pdf/2101.05742
    dt = 0.75
    grid = np.arange(1, p_layers + 1) - 0.5
    init_params = np.concatenate((1 - grid * dt / p_layers, grid * dt / p_layers))

    # `train` minimizes the cost operator, note the -1 prefactor
    training_result = trainer.train(sum(-1/len(cost_ops) * hc for hc in cost_ops).simplify(), init_params)
    params = training_result['optimized_params']
    betas = {f"β[{i}]": param for i, param in enumerate(params[:len(params)//2])}
    gammas = {f"γ[{i}]": param for i, param in enumerate(params[len(params)//2:])}
    params = betas | gammas
    ansatz_qc = qc.assign_parameters(params)
print("QAOA parameters", params)

### 2. QPU上で実行されるようにAnsatz回路を最適化する

アンサッツ回路を実行する前に、指定された量子コンピュータように量子回路をトランスパイルする必要があります。これには1~2分かかることもあります。


In [ ]:
# 実際のデバイスで実行する前に、まず回路をトランスパイルする必要があります！
# ネイティブのheavy-hex graphsのスワップ戦略と色付けなど、トランスパイルのさらなる改善については、
# https://github.com/qiskit-community/qopt-best-practices/tree/main/how_tos を参照してください。
edge_coloring = nx.greedy_color(
    nx.line_graph(moo_graphs[0]), strategy="saturation_largest_first"
)
edge_coloring.update({(k[1], k[0]): v for k, v in edge_coloring.items()})

num_colors = len(set(edge_coloring.values()))

# ここではハードウェアネイティブのグラフがあるので、空のスワップ戦略を作成します。
cmap = CouplingMap(moo_graphs[0].edges())
cmap.make_symmetric()

swap_strategy = SwapStrategy(cmap, ())  # no SWAPs needed

staged_pm = generate_preset_qaoa_pass_manager(backend, swap_strategy, initial_layout=None, edge_coloring=edge_coloring)
# 4. 上記で定義したカスタムパスを使用してQiskitトランスパイルを実行します。
isa_qc = staged_pm.run(ansatz_qc)
naive_isa_qc = generate_preset_pass_manager(backend).run(UnrollBoxes()(ansatz_qc))


print("Two-qubit gate depth", isa_qc.depth(lambda x: len(x.qubits)>1))
print("Number of two-qubit gates", isa_qc.num_nonlocal_gates())
print("Two-qubit gate depth - Naive", naive_isa_qc.depth(lambda x: len(x.qubits)>1))
print("Number of two-qubit gates - Naive", naive_isa_qc.num_nonlocal_gates())

### 3. パレートフロントに沿った点を探るためのアンサッツ回路からのサンプル [10点]

最適化されたアンサッツ回路ができたので、そこから目的加重ベクトルの集合 $c$ を使ってサンプリングし、そこから実現可能なサンプルの集合を決定すればよいです。今のところQiskit Aerの状態ベクトルシミュレータを使っています。実際のQPUで動作する場合は`SamplerV2`を変更するだけで済みます。
次のセルは最大~5分間稼働し、線形計画法を解くことで個々の最大カット問題の緩い上限と下限を決定します。

In [ ]:
# how many samples should be drawn
n_samples = 100
# how many shots per sample
shots = 100

MOOに対するWSMアプローチは、ランダムな重みベクトルがd次元シンプレックス上で均一に分布している場合にパレートフロントを最もよく探求できます。
一方で、正規分布や一様分布から直接サンプリングし、その後に正規化（リスケーリング）を行う方法では、分布の中心付近にサンプルが集中したり、偏りが生じたりする可能性があります。
したがって、次の課題として、`n_obj` 次元の点を `n_samples` 個生成する関数 `gen_cvecs` を実装します。

In [ ]:
# n_obj次元cベクトルに対してランダムなn_samplesを抽出します
# 一様分布または正規分布（その後正規化）から抽出することに注意してください
# 合計が1になる正の重みを取得すると、中心付近に値が集中します
def gen_cvecs(n_samples, n_obj=3):
    c_vecs = np.zeros(shape=(n_samples, n_obj))
    # YOUR CODE HERE, generate n_samples random n_obj-dimensional points    
    return c_vecs

c_vecs = gen_cvecs(n_samples=n_samples, n_obj=n_obj)

In [ ]:
grade_lab5_ex1(gen_cvecs)

In [ ]:

# set error mitigation options for the sampler
options = SamplerOptions()
options.max_execution_time = 600
sampler = SamplerV2(mode=backend, options=options)
    
job = sampler.run([(isa_qc, c_vecs)], shots=shots)
print("Submitted job to", backend.name, "with id", job.job_id(), "and the following options")
sampler.options

ジョブ完了後にQPUから結果を取得します。

In [ ]:
sample_res = job.result()
sample_counts = [sample_res[0].data.meas.get_counts(s) for s in range(n_samples)] 

### Step 4. 解析サンプルからパレートフロントを決定する古典的後処理 [5点]

QAOA に基づく重み付き和法（WSM）アプローチは、QAOA のアンサッツ回路から生成されたサンプルによって誘導される ハイパーボリュームを算出することで評価できます。ここで注意すべき点として、大量のデータを効率的に処理する必要があります。例えば、[Quantum approximate multi-objective optimization](https://www.nature.com/articles/s43588-025-00873-y) に示されている最先端の手法では、3 つの Max-Cut 目的を表す 42 量子ビット回路に対して、5000 サンプル × 各 5000 ショットを処理し、合計 2500 万個の多目的最適化（MOO）解を生成しています。
非効率な後処理を行った場合、この処理には 約 40 分を要します。一方で、次の 3 つのセルに実装されている最適化された後処理では、2500 万個の MOO サンプルに対して およそ 4 分で処理を完了できます。

まず最初に、測定されたビット列を適切な形式に変換する必要があります。すなわち、測定結果を表す文字列の集合を、ブール値からなるコンパクトな配列へと変換します。

In [ ]:
# ここでは、まず文字列を新しい格納場所にコピーし、それを ASCII 文字として解釈します。このとき、文字 '0' は値 48、文字 '1' は値 49 に対応します。その後、それぞれの値から 48 を引くことで、ブール値（Boolean 値）の配列に変換します。
bitstring_length = len(next(iter(sample_counts[0].keys())))bitstring_length = len(next(iter(sample_counts[0].keys())))
x = np.fromiter((np.frombuffer(k.encode("ascii"), dtype="u1") - 48 for i in range(n_samples) for k in sample_counts[i].keys()), dtype=np.dtype((np.uint8, bitstring_length)))

次に、`x`として変数割り当てが与えられた各グラフの最大カット値を求める必要があります。

In [ ]:
# 各グラフの隣接行列を取得する
adj_m = [nx.adjacency_matrix(moo_graphs[i]).toarray() for i in range(n_obj)]
# 各入力グラフの最大カット値を計算します。`x` はブール配列なので `1-x` はその補数となり、`x`adj_m`(1-x)` はカット全体のエッジ（とその重み）を正確に選択します。
fis = np.stack([np.sum((x @ adj_m[i]) * (1 - x), axis=1) for i in range(n_obj)], axis=1)
# 各サンプルを1回だけ処理する
fis = np.unique(fis, axis=0)

最後に、QAOAアプローチのMOOパフォーマンス解析のために、より大きなサンプルセットの超体積を算出します。

In [ ]:
# ハイパーボリュームを計算するための基準点を設定する
hv_ref_point = np.array(lower_bound)
print("Reference Point", hv_ref_point)

num_batches = 1000 if len(fis) > 10000 else len(fis)//10
batch_size = int(len(fis)/num_batches)
hvs = []
for i in range(batch_size, len(fis), batch_size):
    # 外部ライブラリを使用してパレートフロントとそのハイパーボリュームを決定する
    hv = hypervolume(fis[:i], ref=hv_ref_point, maximise=True)
    hvs.append(hv)

# 最後のハイパーボリューム計算にすべてのサンプルを含める
hvs.append(hypervolume(fis, ref=hv_ref_point, maximise=True))


# パレートフロントはハイパーボリューム計算中に決定されるので、ここでは明示的に割り当てる
pareto_front = filter_dominated(fis)
print("Hypervolume induced by all samples", hvs[-1])
print("The Pareto front is defined by", len(pareto_front), "points.")
print("A point of the Pareto front is", pareto_front[0])

In [ ]:
grade_lab5_ex2(max(hvs))

このチャレンジの第一部をクリアし、最先端のアプローチを再現したこと、おめでとうございます!
最近発表された論文では、25 Mioサンプルが必要だったため、超体積は~43000に達しました。残りのチャレンジを楽しんでください!

ここで、このノートブックの冒頭で定義した多目的最大カット問題のソリューション (サンプル) を QPU がどんどん計算するにつれて、ハイパーボリュームがいかに増加するかを実際に確認できます。

In [ ]:
# 10kHzのサンプリングレートを想定
qpu_freq = 10000
time_per_sample = batch_size/qpu_freq
max_hvs = [hvs[i] for i in range(len(hvs))]
seconds = [time_per_sample*(i+1) for i in range(len(hvs))]

plt.plot(seconds, max_hvs)
plt.xlabel("QPU runtime [in seconds]")
plt.ylabel("Hypervolume");

以下の図は、[state-of-the-art paper](https://www.nature.com/articles/s43588-025-00873-y) 論文から引用したもので、本チャレンジで扱っているのと同じグラフ集合に対する多目的最適化（MOO）の解の品質を示しています。
縦軸はハイパーボリュームの品質指標を表しており、オレンジ色の点線は達成可能な最大ハイパーボリュームを示しています。横軸は計算時間（秒）を表しており、QAOA を量子プロセッサ（QPU、ibm_fez）またはシミュレータ（MPS sim.）で実行した場合の実行時間と、$\epsilon$-CM、DCM、DPA-a と呼ばれる 3 つの古典的手法の CPU 実行時間が比較されています。
図から分かるように、提案されている QAOA を用いた MOO アプローチは、QPU の実行時間のみを考慮した場合、解の品質および実行時間の両面において、古典的手法と競争力のある性能を示しています。

In [ ]:
from IPython.display import Image, display
display(Image(filename='figures/paper_comarison.png', width=500))

## Challenge

このチャレンジは、参加者が自分の解答を採点システムに提出するための grader セル で終了します。
採点は、まず自動的なサニティチェック（基本的な妥当性確認）を行い、その後、上位提出物に対して手動チェックが実施されます。
このチャレンジでは、提出したサンプルによって誘起されるハイパーボリューム に応じてポイントが与えられます。

#### ヒント
本チャレンジで上位に入る解答は、以下のような点での工夫や改良が含まれると期待しています:
- QAOA ansatz: 目的関数の重み付き和から解をサンプリングする、より良い方法があるかもしれません。
QAOA レイヤー数（上のコード中の `p_layers`）を変更することで、ノイズの増加と最適解への近さのトレードオフ を調整できます。
- QAOA parameters: 現在保存されている QAOA パラメータは粗いシミュレーションによって求められています。より細かくチューニングすることで、より高品質なサンプルが得られ、計算時間の短縮やハイパーボリュームの向上 が期待できます。より良いパラメータの決定方法の一例は、
[Optimization with light-cones](https://github.com/qiskit-community/qaoa_training_pipeline/blob/main/how_tos/light_cone_optimization.ipynb) の how-to ノートブックに紹介されていますが、このリポジトリには他の手法もあります。
- Transpilation: 本チャレンジのグラフ構造は規則的であるため、構造特化型のトランスパイル手法 によって改善が見込めます。例として、[StagedPassManager を用いた最適 QAOA トランスパイル](https://github.com/qiskit-community/qopt-best-practices/blob/main/how_tos/how_to_apply_optimal_qaoa_transpilation.ipynb)が挙げられます。この手法は上記の 42 量子ビット例でも使用されていますが、入力グラフの 3-正則構造に合わせてスワップ戦略を調整する必要があります。
- Quantum error mitigation: QPU 上で量子回路を実行すると、必ず誤差が発生します。このノートブックの QMOO 手法はもともとノイズ耐性がありますが、サンプル品質を改善することで、より効率的・効果的に解を見つけられる可能性があります。例えば、SQD（sampling-based quantum diagonalization）の configuration recovery に似たローカル探索やポストセレクション手法などが有効かもしれません。
- Post-processing: このチャレンジのグラフ問題に対して パレートフロントをより効率的に探索する方法 があるかもしれません。
ただし、エラー緩和やポスト処理を設計する際に、MOO 問題の解に関する事前情報を仮定しないよう注意してください。

その他の観点での新しいアイデアも大歓迎です。皆さんの提出を楽しみにしています！

### Challenge: 静的な 3 正則 80 ノードグラフ上の Max-Cut［85 点］

In [ ]:
# ここでファイルから読み込んでいる QAOA パラメータは、粗い（簡易的な）シミュレーションによって求められたものなので、さらに改善できる可能性があります。
# 試作・検証用として、12 ノードのグラフが./instances/3_regular_static_12q/ および ./instances/circle_12q/に用意されています。
moo_graphs, qaoa_params, upper_bound, lower_bound = load_problem("./instances/3_regular_static_80q/")

plot_graphs(moo_graphs)

In [ ]:
# <!---------YOUR CODE HERE---------!>
# 1a) Ansatz回路を準備する

In [ ]:
# 1b) Ansatzパラメータをトレーニングするか、qaoa_paramsを使用する
# qaoa_params は、p=1 および p=2 QAOA レイヤーの QAOA \beta および \gamma パラメータを含む辞書です。つまり、{"1": {"\beta[0]": ...}}、"2": {"\beta[0]": ...、"\beta[1]": ...}

In [ ]:
# 2) Transpile the Ansatz circuit [10 pts]
isa_qc = None

In [ ]:
# 次のgraderでは、p = 1 QAOA レイヤーを想定していますが、チャレンジの残りの部分では p > 1 を自由に使用してください。
grade_lab5_ex3(isa_qc)

In [ ]:
# 3) 重みベクトル `c_vecs` を生成し、Ansatz回路からサンプルを取得する

In [ ]:
# 4) サンプルをハイパーボリューム/パレートフロントに後処理する

Grading functionは、次の入力引数を受け取ることを前提としています。

1. あなたの解から得られたハイパーボリューム `max(hvs)`
2. あなたの解によって得られたサンプル `x`
3. 使用した QAOA パラメータの集合 `params`
4. 上記 1 のハイパーボリュームを算出する際に使用したジョブ ID（複数可） `[job.job_id()]`

オプションとして、元のサンプルから新しいサンプル配列を生成する 後処理（post-processing）関数 を提供することもできます。
なお、採点関数の実行には数分かかる場合があります。
また、採点システムはこれらの入力引数の一部をタイムスタンプ付きのファイルとして保存し、チャレンジ上位提出について手動確認を行います（これらのファイルは削除しないでください）。

In [ ]:
# Grader cell
grade_lab_5_ex4(max(hvs), x, isa_qc, params, [job.job_id()])
print("Hypervolume", max(hvs))
print("QPU runtime", max(seconds))

Made with ♥️ by Sebastian Brandhofer, Daniel Egger, and the IBM Quantum Algorithm engineering team! 